In [1]:
import os
import sys
import json
import hashlib
import requests


In [13]:
base_url = "https://web.corral.tacc.utexas.edu/setxuifl/"
from bs4 import BeautifulSoup
import urllib.parse

def get_files_from_url(base_url):
    files = []
    response = requests.get(base_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    BUFFER = 4 * 1073741824

    # Find all links in the page
    for link in soup.find_all('a'):
        href = link.get('href')
        if href:
            # Create absolute URL if relative
            full_url = urllib.parse.urljoin(base_url, href)
            # Skip parent directory links
            if href not in ['../', '../', '/', '#']:
                sha512 = hashlib.sha512()
                length = 0
                
                for chunk in response.iter_content(chunk_size=BUFFER):
                    if chunk:
                        length += len(chunk)
                        sha512.update(chunk)
                
                files.append({
                    "sha512": sha512.hexdigest(),
                    "filename":full_url.split('/')[-1],
                    "url": full_url,
                    "length": length,
                })
    return files

# Example usage
file_list = get_files_from_url(base_url)
for file in file_list:
    print(file)


{'sha512': 'd8a1b8ec314b15819c596d3cd19d27e51176134acde7595d1b6bb51a25a6d2a01c18280e8df12a155e8a7a2d4a3914104526159991591724ecbc0363e501306c', 'filename': '', 'url': 'https://web.corral.tacc.utexas.edu/setxuifl/tropical_cyclones/', 'length': 6615}
{'sha512': 'd8a1b8ec314b15819c596d3cd19d27e51176134acde7595d1b6bb51a25a6d2a01c18280e8df12a155e8a7a2d4a3914104526159991591724ecbc0363e501306c', 'filename': 'tceq_air_monitor_obs_20240528.zip', 'url': 'https://web.corral.tacc.utexas.edu/setxuifl/tceq_air_monitor_obs_20240528.zip', 'length': 6615}
{'sha512': 'd8a1b8ec314b15819c596d3cd19d27e51176134acde7595d1b6bb51a25a6d2a01c18280e8df12a155e8a7a2d4a3914104526159991591724ecbc0363e501306c', 'filename': 'test.txt', 'url': 'https://web.corral.tacc.utexas.edu/setxuifl/test.txt', 'length': 6615}


In [15]:
with open(os.path.join(".", "manifest.json"), "w") as f:
    json.dump(file_list, f, indent=4)


In [ ]:
%%file metadata.toml
# {dset} will be replaced with the dataset name
title="Dataset - SETx-UIFL Datasets {dset}"
author="Will Mobley"
description="a series of datasets for  {dset} component"
date_created="2024-11-8"